In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/efficientnetv2/keras/efficientnetv2_b0_imagenet/2/config.json
/kaggle/input/efficientnetv2/keras/efficientnetv2_b0_imagenet/2/metadata.json
/kaggle/input/efficientnetv2/keras/efficientnetv2_b0_imagenet/2/model.weights.h5
/kaggle/input/kalpesh1/train(1).xlsx
/kaggle/input/img-data/images/9273.png
/kaggle/input/img-data/images/14127.png
/kaggle/input/img-data/images/9292.png
/kaggle/input/img-data/images/11396.png
/kaggle/input/img-data/images/14260.png
/kaggle/input/img-data/images/4353.png
/kaggle/input/img-data/images/7968.png
/kaggle/input/img-data/images/15834.png
/kaggle/input/img-data/images/6490.png
/kaggle/input/img-data/images/5511.png
/kaggle/input/img-data/images/6262.png
/kaggle/input/img-data/images/2664.png
/kaggle/input/img-data/images/12329.png
/kaggle/input/img-data/images/8419.png
/kaggle/input/img-data/images/9703.png
/kaggle/input/img-data/images/13412.png
/kaggle/input/img-data/images/14325.png
/kaggle/input/img-data/images/15055.png
/kaggle/input/img-

In [11]:
# =========================
# CORE LIBRARIES
# =========================
import numpy as np
import pandas as pd

# =========================
# TENSORFLOW & KERAS
# =========================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# =========================
# KERAS CV (for EfficientNetV2)
# =========================
import keras_cv

# =========================
# SKLEARN
# =========================
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# =========================
# OPTIONAL (GOOD PRACTICE)
# =========================
import os


In [12]:
df=pd.read_excel('/kaggle/input/kalpesh1/train(1).xlsx')

In [13]:
df['date'] = pd.to_datetime(df['date'])

In [14]:
df['year_sold']=df['date'].dt.year

In [15]:
df['HouseAge'] = df['year_sold'] - df['yr_built']

**IMAGE PATH CREATION**

In [16]:
IMAGE_DIR = "/kaggle/input/img-data/images"   # adjust if needed

df["image_path"] = df.index.astype(str).map(
    lambda x: os.path.join(IMAGE_DIR, f"{x}.png")
)

print("Sample image paths:")
print(df["image_path"].head(5).tolist())

exists_count = df["image_path"].apply(os.path.exists).sum()
print("Images found on disk:", exists_count, "/", len(df))


Sample image paths:
['/kaggle/input/img-data/images/0.png', '/kaggle/input/img-data/images/1.png', '/kaggle/input/img-data/images/2.png', '/kaggle/input/img-data/images/3.png', '/kaggle/input/img-data/images/4.png']
Images found on disk: 16209 / 16209


**TABULAR FEATURES AND TARGET VARIABLE DECLARATION**

In [17]:
# =========================
# TABULAR FEATURES (18 cols)
# =========================
tabular_features = [
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'zipcode',
    'lat',
    'long',
    'sqft_living15',
    'sqft_lot15',
    'HouseAge'
]

# =========================
# TARGET
# =========================
target = 'price'




**IMAGE LOADING FROM THE DATASET**

In [18]:
IMG_SIZE = (224, 224)

def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img / 255.0, tf.float32)
    return img

**DATA SPLIT (TAB + IMG)**

In [19]:
# =========================
# 2) DATA SPLIT (TAB + IMG)
# =========================
X_tabular = df[tabular_features]      # list of your 18 tabular columns
X_image_paths = df['image_path']      # image paths
y = df[target]                        # target column

X_tab_train, X_tab_test, X_img_train, X_img_test, y_train, y_test = train_test_split(
    X_tabular,
    X_image_paths,
    y,
    test_size=0.2,
    random_state=42
)


In [20]:
# =========================
# QUICK SANITY CHECK
# =========================
print("Tabular shape:", X_tabular.shape)
print("Image path sample:", X_image_paths.iloc[0])
print("Target sample:", y.iloc[0])


Tabular shape: (16209, 18)
Image path sample: /kaggle/input/img-data/images/0.png
Target sample: 268643


**TABULAR MODEL**

In [21]:
tabular_input = keras.layers.Input(
    shape=(X_tabular.shape[1],),
    name='tabular_input'
)

x_tab = keras.layers.Dense(32, activation='relu')(tabular_input)

tabular_model = keras.Model(
    inputs=tabular_input,
    outputs=x_tab,
    name="tabular_model"
)

tabular_model.summary()

I0000 00:00:1767711859.430957      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "tabular_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tabular_input (InputLayer)      │ (None, 18)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           608 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 608 (2.38 KB)

 Trainable params: 608 (2.38 KB)

 Non-trainable params: 0 (0.00 B)

**IMAGE MODEL**

In [22]:
image_input = keras.layers.Input(shape=(*IMG_SIZE, 3), name='image_input')

efficientnet = keras_cv.models.EfficientNetV2Backbone.from_preset(
    "efficientnetv2_b0_imagenet"
)

x_image = efficientnet(image_input)
x_image = keras.layers.GlobalAveragePooling2D()(x_image)
x_image = keras.layers.Dense(32, activation='relu')(x_image)

image_model = keras.Model(
    inputs=image_input,
    outputs=x_image,
    name="image_model"
)

image_model.summary()


Model: "image_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficient_net_v2b0_backbone     │ (None, 7, 7, 1280)     │     5,919,312 │
│ (EfficientNetV2Backbone)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │        40,992 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,960,304 (22.74 MB)

 Trainable params: 5,899,696 (22.51 MB)

 Non-trainable params: 60,608 (236.75 KB)

**MULTIMODAL FUSION**

In [23]:
combined = keras.layers.concatenate(
    [tabular_model.output, image_model.output]
)

x = keras.layers.Dense(32, activation='relu')(combined)
output = keras.layers.Dense(1, activation='linear', name='Price')(x)

model = keras.Model(
    inputs={
        'tabular_input': tabular_model.input,
        'image_input': image_model.input
    },
    outputs=output
)

model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficient_net_v2b0… │ (None, 7, 7,      │  5,919,312 │ image_input[0][0] │
│ (EfficientNetV2Bac… │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tabular_input       │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ efficient_net_v2… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        608 │ tabular_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │     40,992 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      2,080 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Price (Dense)       │ (None, 1)         │         33 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,963,025 (22.75 MB)

 Trainable params: 5,902,417 (22.52 MB)

 Non-trainable params: 60,608 (236.75 KB)

**COMPILE**

In [24]:
model.compile(
    optimizer="adam",
    loss='mean_absolute_error',
    metrics=['mean_absolute_error']
)


**DATA GENERATOR**

In [25]:
def data_generator(tab_data, img_paths, labels, batch_size=32, shuffle_buffer=1024):

    def parse_function(tab_input, img_path, label):
        img_input = load_and_preprocess_image(img_path)
        return {
            'tabular_input': tab_input,
            'image_input': img_input
        }, label

    tab_data = tab_data.to_numpy()
    img_paths = img_paths.to_numpy()
    labels = labels.to_numpy()

    dataset = tf.data.Dataset.from_tensor_slices(
        (tab_data, img_paths, labels)
    )

    return (
        dataset
        .shuffle(buffer_size=shuffle_buffer)
        .map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )


batch_size = 8
train_dataset = data_generator(X_tab_train, X_img_train, y_train, batch_size)
test_dataset  = data_generator(X_tab_test,  X_img_test,  y_test,  batch_size)



**TRAINING OF MULTIMODAL**

In [26]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5
)


Epoch 1/5


I0000 00:00:1767711901.262756     206 service.cc:152] XLA service 0x7e2d8c002260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767711901.262792     206 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1767711908.088968     206 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1767711938.996332     206 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1621/1621 ━━━━━━━━━━━━━━━━━━━━ 177s 64ms/step - loss: 260801.9688 - mean_absolute_error: 260801.9688 - val_loss: 196091.9375 - val_mean_absolute_error: 196091.9375
Epoch 2/5
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 60s 37ms/step - loss: 164414.0781 - mean_absolute_error: 164414.0781 - val_loss: 125417.7734 - val_mean_absolute_error: 125417.7734
Epoch 3/5
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 60s 37ms/step - loss: 129574.2969 - mean_absolute_error: 129574.2969 - val_loss: 123534.7344 - val_mean_absolute_error: 123534.7344
Epoch 4/5
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 60s 37ms/step - loss: 124272.9844 - mean_absolute_error: 124272.9844 - val_loss: 121095.5938 - val_mean_absolute_error: 121095.5938
Epoch 5/5
1621/1621 ━━━━━━━━━━━━━━━━━━━━ 60s 37ms/step - loss: 119765.2344 - mean_absolute_error: 119765.2344 - val_loss: 112738.3125 - val_mean_absolute_error: 112738.3125


**EVALUATION (MAE + R2)**

In [27]:
from sklearn.metrics import r2_score
import numpy as np

loss, mae = model.evaluate(test_dataset)
print(f"Test MAE: {mae}")

y_true = []
y_pred = []

for batch in test_dataset:
    inputs, labels = batch
    preds = model.predict(inputs, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(preds.squeeze())

r2 = r2_score(y_true, y_pred)
print(f"Test R2 Score: {r2}")


406/406 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 111239.5547 - mean_absolute_error: 111239.5547
Test MAE: 112738.25
Test R2 Score: 0.6858253647266186


In [28]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"Test RMSE: {rmse}")


Test RMSE: 198558.14859173654
